# Waveform Analysis Exploration 

Need a function for mass data addition (all the unique EDF files). I'll merge them into a single table for easy analysis. Waves are waves-- so long as they have the same filtering and they're all originally raw EEG! 

In [1]:
import numpy as np
import pandas as pd
import mne 
from mne.time_frequency import psd_array_welch
import matplotlib.pyplot as plt

def process_edf_pair(psg_path, hypnogram_path): 
    '''
    Loads EDF EEG recording and corresponding hypnogram 
    Extracts 30s epochs, labels them, and returns a features df

    Returns: 
        features (pd.DataFrame): Features + newly created 'label' col (sleep stage as str)
    '''

    raw = mne.io.read_raw_edf(psg_path, preload=True) 
    raw.set_annotations(mne.read_annotations(hypnogram_path))
    raw.pick(['EEG Fpz-Cz']) 

    events, event_id = mne.events_from_annotations(raw)
    epochs = mne.Epochs(raw, events=events, event_id=event_id,
                        tmin=0, tmax=30, baseline=None, preload=True)
    
    # Label mapping (using standardidized strings)
    reverse_event_map = {v: k for k, v in event_id.items()} 

    stage_map = {
        'Sleep stage W': 'Wake',
        'Sleep stage 1': 'N1',
        'Sleep stage 2': 'N2',
        'Sleep stage 3': 'N3',
        'Sleep stage 4': 'N3',
        'Sleep stage R': 'REM'
    }

    labels = []
    for e in epochs.events:
        label = stage_map.get(reverse_event_map.get(e[-1]), None) 
        labels.append(label if label is not None else 'UNKNOWN')

    # Filtering unknowns
    valid_mask = [label != 'UNKNOWN' for label in labels]
    epochs = epochs[valid_mask]
    labels = [l for l in labels if l != 'UNKNOWN']

    # Feat Extraction
    data = epochs.get_data()
    sfreq = epochs.info['sfreq'] 
    psds, freqs = psd_array_welch(data, sfreq=sfreq, fmin=0.5, fmax=40, n_fft=256)

    features = pd.DataFrame({
        'delta': bandpower(psds, freqs, (0.5, 4)),
        'theta': bandpower(psds, freqs, (4, 8)),
        'alpha': bandpower(psds, freqs, (8, 13)),
        'beta':  bandpower(psds, freqs, (13, 30)),
        'label': labels
    })

    return features

def bandpower(psds, freqs, band):
    '''
    Small hepler for process_edf_pair
    Returns mean band power in specified frequency range
    '''
    idx = (freqs >= band[0]) & (freqs <= band[1])
    return psds[:, :, idx].mean(axis=-1).mean(axis=1)



### Now Loop over all .edf files!

In [ ]:
import os

# Match PSG and Hypnogram files by prefix
data_dir = '../data/sleep_waves/'
files = os.listdir(data_dir) 

# Grouping PSG and Hypnogram files by shared sybject root (first 7 chars) 
psg_files = [f for f in files if f.endswith('-PSG.edf')]
hyp_files = [f for f in files if 'Hypnogram' in f] 

file_pairs = []

for psg in psg_files:
    subj_root = psg[:7] # example: 'ST7011J'
    matching_hyp = [h for h in hyp_files if h.startswith(subj_root)]

    if matching_hyp: 
        psg_path = os.path.join(data_dir, psg) 
        hyp_path = os.path.join(data_dir, matching_hyp[0]) # Note: assuming only 1 match! 
        file_pairs.append((psg_path, hyp_path)) 
    else: 
        print(f'No matching hypnogram for {psg}')

print(f'Found {len(file_pairs)} valid PSG-Hypnogram edf pairs') 

# Process all and concat 
all_feats = []

for psg, hyp in file_pairs:
    print(f"🧪 Processing: {os.path.basename(psg)} & {os.path.basename(hyp)}")
    try: 
        features = process_edf_pair(psg, hyp)
        all_feats.append(features) 
        print(f'\n✅ Processed: {os.path.basename(psg)}:{len(features)} epochs\n')
    except Exception as e: 
        print(f'❌ Failed on {psg}: {e}')

df_edf = pd.concat(all_feats, ignore_index=True) 




In [3]:
df_edf.to_csv('../data/eeg_hypno.csv', index=False)

df_edf

,delta,theta,alpha,beta,label
0,6.160646e-12,3.309411e-12,1.943593e-12,2.366059e-12,Wake
1,6.851487e-12,2.188516e-12,1.104633e-12,8.372303e-13,N1
2,2.707088e-11,3.864857e-12,1.452854e-12,8.678398e-13,N2
3,2.403117e-11,1.126442e-12,1.835699e-12,5.003665e-13,N1
4,4.395439e-11,2.545034e-12,1.276473e-12,8.361875e-13,N2
...,...,...,...,...,...
6021,1.416674e-11,4.520787e-12,1.160433e-12,2.291765e-13,REM
6022,4.827426e-10,7.670029e-12,3.671621e-12,3.220177e-12,Wake
6023,5.953652e-12,1.894862e-12,1.252515e-12,5.406972e-13,N1
6024,1.037589e-11,2.498837e-12,8.997023e-13,2.299988e-13,N2


### Training a Basic Classifier (RF)

In [4]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report 

X = df_edf.drop(columns=['label'])
y = df_edf['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=10)

clf = RandomForestClassifier(n_estimators=100, random_state=10, class_weight='balanced')
clf.fit(X_train, y_train) 

y_pred = clf.predict(X_test) 
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

          N1       0.57      0.56      0.56       270
          N2       0.54      0.58      0.56       359
          N3       0.71      0.78      0.74       346
         REM       0.60      0.38      0.47        78
        Wake       0.62      0.50      0.55       153

    accuracy                           0.61      1206
   macro avg       0.61      0.56      0.58      1206
weighted avg       0.61      0.61      0.61      1206



### Pipeline Project Adapted to Waveforms:

In [5]:
import sys
import os

from sklearn.calibration import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

sys.path.append(os.path.abspath(".."))

from src.run_pipeline import tune_and_train_full

# Label Encoding with decoding built into tune_and_train_full()
le = LabelEncoder()
y_encoded = le.fit_transform(y) 
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, stratify=y_encoded, random_state=10)

rf_wave, best_params = tune_and_train_full(RandomForestClassifier, 
                    'Random Forest', 
                    X_train, 
                    y_train,
                    X_test=X_test, 
                    y_test=y_test, 
                    model_params={'class_weight': 'balanced'},
                    sample_frac=0.95, 
                    scoring='accuracy',
                    label_encoder=le) 

🔍 Sampled 4293 rows (95.0%)
Class distribution:
1    1277
2    1232
0     961
4     543
3     280

Starting Grand Tuner | CV: 5 | Scoring: accuracy
Model: RandomForestClassifier | SMOTE: True | Random Iterations: 20

**RandomForestClassifier param grid is None, using default

Fitting 5 folds for each of 20 candidates, totalling 100 fits
⏱️ RandomizedSearchCV completed in 0.47 minutes

Fitting 5 folds for each of 9 candidates, totalling 45 fits
⏱️ GridSearchCV completed in 0.52 minutes

🏆 Best Hyperparameters: {
  "classifier__n_estimators": 250,
  "classifier__min_samples_split": 5,
  "classifier__min_samples_leaf": 2,
  "classifier__max_features": "sqrt",
  "classifier__max_depth": 10
}

📊 Best accuracy: 0.5635
💾 Saved best params for RandomForestClassifier to ../tuned_params\RandomForestClassifier_best_params.json


[TRAINING] Starting model training...

→ Training Random Forest with params:
{
    "class_weight": "balanced",
    "classifier__n_estimators": 250,
    "classifier__min_s

## Great, Classifier is Built! What Now?
Just completed work on a sleep stage classifier that is ~60% effective. While that's not bad... we can do better, by using the elements of the data to our advantage: 
- Sleep is **not** random, it follows structured stage transitions
- HMM can capture those structured transitions as probabilities
- We can use a HMM approach to smooth the pipeline model's predictions into a more biologically probably sequence.


### While Discussing Goals: 
**Real-time sleep classification**. It's possible (with a ton of work). That is our final product. 

<hr>

### Back to Reality: HMM Smoothing Pass 

It's important to remember that the HMM is *not* making new predictions or replacing the already nicely trained and optimized classifier. The HMM will function to refine the outputs.  
  
We'll learn from the data in order to estimate stage transition probabilities. For example: how likely are you to stay in N2, or jump to N3, etc. 

In [23]:
# Step 1. 
# Convert predicted and true labels back to class indices

from sklearn.preprocessing import LabelEncoder

y_test_decoded = le.inverse_transform(y_test) 
y_pred_decoded = le.inverse_transform(rf_wave.predict(X_test)) 

# Re-encode with a fresh label encoder for HMM
le_hmm = LabelEncoder()
y_test_idx = le_hmm.fit_transform(y_test_decoded)
y_pred_idx = le_hmm.fit_transform(y_pred_decoded) 

In [29]:
# Step 2.
# Fit the HMM (only on stage transitions)

from hmmlearn import hmm
import numpy as np

n_states = len(le_hmm.classes_) 
model_hmm = hmm.MultinomialHMM(n_components=n_states, n_iter=100, tol=0.01, verbose=True) 

# Now reshape to (n_samples, 1)-- required by hmmlearn
y_test_seq = y_test_idx.reshape(-1, 1) 

# Fit transition and start probs using the true label seq
model_hmm.fit(y_test_seq) 

         1       0.00000000             +nan
         2       0.00000000      +0.00000000


MultinomialHMM(n_components=5, n_iter=100,
               n_trials=array([3, 2, 0, ..., 1, 2, 2]),
               random_state=RandomState(MT19937) at 0x187FF33E440,
               verbose=True)

In [26]:
# Step 3. 
# Use Classifier preds + HMM for smoothing

# Classifier pred sequence (unsmoothed) 
y_pred_seq = y_pred_idx.reshape(-1, 1) 

# Applying Viterbi to get smoothed predictions
smoothed_idx = model_hmm.predict(y_pred_seq) 

# Decode to original class labels
smoothed_labels = le_hmm.inverse_transform(smoothed_idx) 

To make things more clear, we now have: 
- `y_test_decoded` -- ground truth
- `y_pred_decoded` -- raw classifier 
- `smoothed_labels` -- post-HMM output

In [27]:
# Step 4. 
# Evaluate Improvements

from sklearn.metrics import classification_report

print("🔍 Classifier (Raw) Performance:")
print(classification_report(y_test_decoded, y_pred_decoded))

print("🧠 HMM-Smooth Performance:")
print(classification_report(y_test_decoded, smoothed_labels))


🔍 Classifier (Raw) Performance:
              precision    recall  f1-score   support

          N1       0.56      0.54      0.55       337
          N2       0.56      0.49      0.52       449
          N3       0.71      0.81      0.76       432
         REM       0.50      0.55      0.52        98
        Wake       0.54      0.54      0.54       191

    accuracy                           0.60      1507
   macro avg       0.57      0.59      0.58      1507
weighted avg       0.60      0.60      0.60      1507

🧠 HMM-Smooth Performance:
              precision    recall  f1-score   support

          N1       0.22      1.00      0.37       337
          N2       0.00      0.00      0.00       449
          N3       0.00      0.00      0.00       432
         REM       0.00      0.00      0.00        98
        Wake       0.00      0.00      0.00       191

    accuracy                           0.22      1507
   macro avg       0.04      0.20      0.07      1507
weighted avg       

c:\Users\andre\OneDrive\Desktop\Code_Projects\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\andre\OneDrive\Desktop\Code_Projects\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\andre\OneDrive\Desktop\Code_Projects\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f

Leaving things here for tonight, tomorrow *morning* come back to create and work on the visualization code...

In [ ]:
import matplotlib.pyplot as plt
import numpy as np 

def plot_sleep_sequences(y_true, y_pred, y_hmm, label_order=None): 
    '''
    Plots 1) ground truth, 2) classifier preds, and 3) HMM-smoothed preds over time

    Args: 
        y_true (array-like): Ground truth labels
        y_preds (array-like): Classifier raw predictions
        y_hmm (array-like): HMM-smoothed predictions
        label_order (list): Optional, ordered list of sleep stage names for consistent y-axis
    '''

